<a href="https://colab.research.google.com/github/EricRoh-kr/st9_youtubevid/blob/main/st9_youtubevid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# 필요한 라이브러리 설치
import subprocess
import sys
import os
import zipfile
import shutil
from datetime import datetime

# Google Colab 환경 확인 및 위젯/HTML 표시 라이브러리 로드
try:
    from google.colab import files
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# --- 실행 잠금 장치 (전역 변수) ---
IS_RUNNING = False

# --- 핵심 기능 함수 ---

def install_ytdlp(output_widget):
    """yt-dlp 라이브러리를 설치하거나 업그레이드합니다."""
    with output_widget:
        try:
            print("yt-dlp 설치/업그레이드 중...")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "-U", "yt-dlp"],
                stdout=subprocess.PIPE, stderr=subprocess.PIPE
            )
            print("✅ yt-dlp가 준비되었습니다.")
            return True
        except subprocess.CalledProcessError:
            print("❌ yt-dlp 설치에 실패했습니다.")
            return False

def create_download_folder():
    """다운로드 폴더를 생성하고, 이미 있다면 정리합니다."""
    folder_name = "영상저장함"
    if os.path.exists(folder_name):
        shutil.rmtree(folder_name)
    os.makedirs(folder_name)
    return folder_name

def download_videos_to_folder(urls, quality, folder_name, output_widget):
    """영상들을 지정된 폴더에만 저장하고, 진행 상황을 출력합니다."""
    with output_widget:
        print(f"⬇️ 영상들을 폴더에 저장합니다... (총 {len(urls)}개)")
        print("="*50)

        if quality == '고화질 (최고 품질, 용량 큼)':
            format_option = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best"
            print("✨ 고화질 모드로 저장합니다.")
        else:
            format_option = "best[height<=720]/best"
            print("👍 일반화질 모드로 저장합니다.")

        success_count = 0
        failed_urls = []

        for i, url in enumerate(urls, 1):
            if not url.strip():
                continue

            print(f"\n[{i}/{len(urls)}] 처리 중: {url}", end="")
            try:
                output_template = os.path.join(folder_name, "%(title)s.%(ext)s")
                cmd = ["yt-dlp", "-f", format_option, "-o", output_template, "--no-warnings", url]
                result = subprocess.run(cmd, capture_output=True, text=True, encoding="utf-8")

                if result.returncode == 0:
                    print(" ✅ 저장 완료!")
                    success_count += 1
                else:
                    print(" ❌ 저장 실패!")
                    print(f"   [오류 원인]: {result.stderr.strip()}")
                    failed_urls.append(url)

            except Exception as e:
                print(f" ❌ 처리 중 예외 발생: {e}")
                failed_urls.append(url)

        print("\n" + "="*50)
        print("🎉 폴더 저장 결과 🎉")
        print(f"✅ 성공: {success_count}개")
        print(f"❌ 실패: {len(failed_urls)}개")

        if failed_urls:
            print("\n❌ 실패한 링크:")
            for url in failed_urls:
                print(f"   - {url}")

        return success_count > 0

def create_and_download_zip(folder_name, output_widget):
    """폴더를 ZIP으로 압축하고 Colab에서 자동 다운로드합니다."""
    with output_widget:
        today = datetime.now().strftime("%m%d")
        zip_filename = f"{today}_유튜브다운로드_{datetime.now().strftime('%H%M%S')}.zip"

        print("\n" + "="*50)
        print(f"🗜️ '{zip_filename}' 파일로 압축 중...")

        try:
            with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=6) as zipf:
                for root, _, files_in_dir in os.walk(folder_name):
                    for file in files_in_dir:
                        file_path = os.path.join(root, file)
                        zipf.write(file_path, os.path.relpath(file_path, folder_name))

            print(f"✅ 압축 완료: {zip_filename}")
            print("⬇️ 브라우저에서 파일 다운로드를 시작합니다...")
            files.download(zip_filename)
            print("👍 다운로드 창이 나타나지 않으면 팝업 차단을 확인해주세요.")

        except Exception as e:
            print(f"❌ ZIP 파일 생성 또는 다운로드 중 오류 발생: {e}")

# --- ipywidgets UI 및 이벤트 핸들러 ---

def on_save_button_clicked(b):
    """[버튼 1] 영상 저장 이벤트 핸들러"""
    global IS_RUNNING
    if IS_RUNNING: return
    IS_RUNNING = True

    save_button.disabled = True
    zip_button.disabled = True
    output_area.clear_output()

    try:
        with output_area:
            urls = urls_input.value.strip().split('\n')
            urls = [url for url in urls if url.strip()]

            if not urls:
                print("❌ 다운로드할 링크를 입력해주세요!")
                return

            if not install_ytdlp(output_area):
                return

            folder_name = create_download_folder()
            has_downloads = download_videos_to_folder(urls, quality_choice.value, folder_name, output_area)

            if has_downloads:
                print("\n✅ 모든 영상이 폴더에 저장되었습니다.")
                print("👉 이제 '2. 압축 파일 다운로드' 버튼을 눌러주세요.")
                zip_button.disabled = False # 다운로드 버튼 활성화
            else:
                print("\n❌ 저장된 영상이 없어 압축을 진행할 수 없습니다.")
    finally:
        IS_RUNNING = False
        save_button.disabled = False

def on_zip_button_clicked(b):
    """[버튼 2] 압축 및 다운로드 이벤트 핸들러"""
    global IS_RUNNING
    if IS_RUNNING: return
    IS_RUNNING = True

    zip_button.disabled = True
    output_area.clear_output()

    try:
        with output_area:
            folder_name = "영상저장함"
            if not os.path.exists(folder_name) or not os.listdir(folder_name):
                print("❌ 저장된 영상 폴더를 찾을 수 없거나 비어있습니다. 먼저 영상을 저장해주세요.")
                return
            create_and_download_zip(folder_name, output_area)
    finally:
        IS_RUNNING = False

# --- 메인 실행 로직 ---

if __name__ == "__main__":
    if IN_COLAB:
        style = {'description_width': 'initial'}
        urls_input = widgets.Textarea(
            placeholder='다운로드할 유튜브 링크를 한 줄에 하나씩 입력하세요.',
            layout=widgets.Layout(width='95%', height='150px')
        )
        quality_choice = widgets.RadioButtons(
            options=['일반화질 (720p 이하, 빠름)', '고화질 (최고 품질, 용량 큼)'],
            description='화질 선택:', style=style
        )
        save_button = widgets.Button(
            description='1. 영상 폴더에 저장하기',
            button_style='primary', icon='save',
            layout=widgets.Layout(width='250px', height='40px')
        )
        zip_button = widgets.Button(
            description='2. 압축 파일 다운로드',
            button_style='success', icon='download',
            layout=widgets.Layout(width='250px', height='40px'),
            disabled=True # 처음에는 비활성화
        )
        output_area = widgets.Output(layout=widgets.Layout(width='95%', border='1px solid grey', padding='10px'))

        save_button.on_click(on_save_button_clicked)
        zip_button.on_click(on_zip_button_clicked)

        ui_title = widgets.HTML("<h1>🎬 YouTube 영상 다운로더 (2단계)</h1>")
        ui_description = widgets.HTML("<p><b>1단계:</b> 링크를 붙여넣고 '영상 폴더에 저장하기'를 누르세요.<br><b>2단계:</b> 저장이 완료되면 활성화되는 '압축 파일 다운로드' 버튼을 눌러 최종 파일을 받으세요.</p>")
        buttons_box = widgets.HBox([save_button, zip_button])

        display(widgets.VBox([
            ui_title, ui_description, urls_input, quality_choice,
            buttons_box, widgets.HTML("<hr><h3>📜 진행 상황</h3>"), output_area
        ]))
    else:
        print("이 스크립트는 Google Colab 환경에서만 작동합니다.")
